In [8]:
from dotenv import load_dotenv
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt

from eth_rpc import PrivateKeyWallet
from emp.orderly.utils import from_address
from emp.orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp.orderly_types import PerpetualAssetType, Interval
from emp.orderly.onboarding.faucet import request_testnet_funds

load_dotenv()

wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)

sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

# get some testnet USDC from the faucet
await request_testnet_funds(wallet)
print(await wallet.balance())

0


In [11]:
class EMSB(Strategy):
    order_size: float = 0.5
    nine_days_length: int = 9
    fourteen_days_length: int = 14
    length: int = 14
    
    @classmethod
    def update_lags(cls, nine_days_length, fourteen_days_length, length):
        cls.nine_days_length = nine_days_length
        cls.fourteen_days_length = fourteen_days_length
        cls.length = length

    def init(self):
        close = self.data.close
        self.ema_nine_days = self.I(EMA, close, self.nine_days_length)
        self.ema_fourteen_days = self.I(EMA, close, self.fourteen_days_length)
        self.slope = self.I(SLOPE, close, self.length)

    def next(self):
        if (
            crossover(self.ema_nine_days, self.ema_fourteen_days) &
            (self.slope > 15)
        ):
            self.position.close()
            self.buy(size=self.order_size)
        elif (
            crossover(self.ema_fourteen_days, self.ema_nine_days) &
            (self.slope < -15)
        ):
            self.position.close()
            self.sell(size=self.order_size)


tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# load strategy and data
tester.set_strategy(EMSB)
await tester.load_data(
    lookback=14,
    interval=Interval.five_minute,
    asset=PerpetualAssetType.BTC,
)
print(tester._history.head())
tester.backtest()
plt.show()

                        open     high      low    close  volume_asset  volume
timestamp                                                                    
2024-09-23 09:15:00  62863.5  62863.5  62863.5  62863.5           0.0     0.0
2024-09-23 09:20:00  62863.5  62863.5  62863.5  62863.5           0.0     0.0
2024-09-23 09:25:00  62863.5  62863.5  62863.5  62863.5           0.0     0.0
2024-09-23 09:30:00  62863.5  62863.5  62863.5  62863.5           0.0     0.0
2024-09-23 09:35:00  62863.5  62863.5  62863.5  62863.5           0.0     0.0


/workspaces/orderly-test-ground/emp/orderly/backtest/backtesting.py:1175: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  self._init_data(self._history)
